In [1]:
from mxnet import gluon
import mxnet as mx

In [2]:
from mxnet import image
from mxnet import nd
import mxnet as mx

data_shape = 448
batch_size = 4
rgb_mean = nd.array([123, 117, 104])
rgb_std = nd.array([58.395, 57.12, 57.375])

def get_iterators(data_shape, batch_size):
    class_names = ['banana','blueberry','grape','peach','pineapple','strawberry']
    num_class = len(class_names)
    train_iter = image.ImageDetIter(
        batch_size=batch_size,
        data_shape=(3, data_shape, data_shape),
        #path_imgrec=data_dir+'train.rec',
        #path_imgidx=data_dir+'train.idx',
        path_imglist='D:/1/test.lst',
        path_root='D:/1/data/',
        shuffle=True,
        mean=True,
        std=True,
        #rand_crop=1,
        min_object_covered=0.95,
        max_attempts=200
    )
    '''
    val_iter = image.ImageDetIter(
        batch_size=batch_size,
        data_shape=(3, data_shape, data_shape),
        path_imgrec=data_dir+'val.rec',
        shuffle=False,
        mean=True,
        std=True)
        '''
    #return train_iter, val_iter, class_names, num_class
    return train_iter, class_names, num_class
#train_data, test_data, class_names, num_class = get_iterators(
#    data_shape, batch_size)
train_data, class_names, num_class = get_iterators(data_shape, batch_size)

In [3]:
import mxnet as mx
data_iter = mx.image.ImageDetIter(batch_size=2, data_shape=(3,1024, 1024),path_imglist='D:/1/test.lst',path_root='D:/1/data/')

In [41]:
data_iter.reset()
for data in data_iter:
    d = data.data[0]
    l = data.label[0]
    print(d.shape)
    print(l[0])
    break 

(1L, 3L, 1024L, 1024L)

[[ 1.          0.17152777  0.33828124  0.31874999  0.42226562]
 [ 1.          0.33958334  0.33867186  0.48819444  0.42265624]
 [ 1.          0.17152777  0.61914062  0.31874999  0.70351565]
 [ 1.          0.3361111   0.61874998  0.4861111   0.70351565]
 [ 0.          0.50416666  0.61874998  0.65555555  0.70156252]
 [ 0.          0.1701389   0.5277344   0.32361111  0.60976565]
 [ 0.          0.50277776  0.52656251  0.65416664  0.60859376]
 [ 0.          0.50208336  0.33828124  0.65416664  0.42109376]
 [ 5.          0.6722222   0.33945313  0.81944442  0.42109376]
 [ 5.          0.34097221  0.43320313  0.48402777  0.51601565]
 [ 5.          0.67083335  0.52539062  0.81805557  0.61171877]
 [ 5.          0.6701389   0.62031251  0.81805557  0.70429689]
 [ 3.          0.17222223  0.43125001  0.31874999  0.51601565]
 [ 3.          0.3361111   0.52460939  0.48333332  0.61093748]
 [ 3.          0.50486112  0.43203124  0.65208334  0.51640624]
 [ 3.          0.67291665  0.43

In [42]:

import numpy as np
import matplotlib.pyplot as plt
idx = 0
img = d[0].asnumpy()  # grab the first image, convert to numpy array
img = img.transpose((1, 2, 0))  # we want channel to be the last dimension #img += np.array([123, 117, 104])
img = img.astype(np.uint8)  # use uint8 (0-255)
for ll in l[0].asnumpy():
    if ll[0]<0: #-1是补充的，不是label
        break
    else:
        print(ll)
        xmin= int(ll[1]*1024) #因为我的x的shape是734
        ymin= int(ll[2]*1024) #因为我的y的shape是804
        xmax= int(ll[3]*1024) 
        ymax = int(ll[4]*1024) 
        rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, edgecolor=(1, 0, 0), linewidth=1)
        plt.gca().add_patch(rect)
        #break
 
plt.imshow(img,'gray')# 我的是灰度图，如果你们是彩色图也可以做别的操作
plt.show()


[ 1.          0.17152777  0.33828124  0.31874999  0.42226562]
[ 1.          0.33958334  0.33867186  0.48819444  0.42265624]
[ 1.          0.17152777  0.61914062  0.31874999  0.70351565]
[ 1.          0.3361111   0.61874998  0.4861111   0.70351565]
[ 0.          0.50416666  0.61874998  0.65555555  0.70156252]
[ 0.          0.1701389   0.5277344   0.32361111  0.60976565]
[ 0.          0.50277776  0.52656251  0.65416664  0.60859376]
[ 0.          0.50208336  0.33828124  0.65416664  0.42109376]
[ 5.          0.6722222   0.33945313  0.81944442  0.42109376]
[ 5.          0.34097221  0.43320313  0.48402777  0.51601565]
[ 5.          0.67083335  0.52539062  0.81805557  0.61171877]
[ 5.          0.6701389   0.62031251  0.81805557  0.70429689]
[ 3.          0.17222223  0.43125001  0.31874999  0.51601565]
[ 3.          0.3361111   0.52460939  0.48333332  0.61093748]
[ 3.          0.50486112  0.43203124  0.65208334  0.51640624]
[ 3.          0.67291665  0.43203124  0.82013887  0.51757812]


In [3]:
from mxnet.gluon import nn
import numpy as np
class YOLO2Output(nn.HybridBlock):
    def __init__(self, num_class, anchor_scales, **kwargs):
        super(YOLO2Output, self).__init__(**kwargs)
        assert num_class > 0, "number of classes should > 0, given {}".format(num_class)
        self._num_class = num_class
        assert isinstance(anchor_scales, (list, tuple)), "list or tuple of anchor scales required"
        assert len(anchor_scales) > 0, "at least one anchor scale required"
        for anchor in anchor_scales:
            assert len(anchor) == 2, "expected each anchor scale to be (width, height), provided {}".format(anchor)
        self._anchor_scales = anchor_scales
        out_channels = len(anchor_scales) * (num_class + 1 + 4)
        with self.name_scope():
            self.output = nn.Conv2D(out_channels, 1, 1)

    def hybrid_forward(self, F, x, *args):
        return self.output(x)


In [4]:
from mxnet.gluon.model_zoo import vision
pretrained = vision.get_model('resnet18_v1', pretrained=True).features
net = nn.HybridSequential()
for i in range(len(pretrained) - 2):
    net.add(pretrained[i])

# anchor scales, try adjust it yourself
scales = [[3.3004, 3.59034],
          [9.84923, 8.23783]]

# use 2 classes, 1 as dummy class, otherwise softmax won't work
predictor = YOLO2Output(num_class, scales)
predictor.initialize()
net.add(predictor)


In [5]:
print net

HybridSequential(
  (0): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
  (2): Activation(relu)
  (3): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
  (4): HybridSequential(
    (0): BasicBlockV1(
      (body): HybridSequential(
        (0): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
        (2): Activation(relu)
        (3): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
      )
    )
    (1): BasicBlockV1(
      (body): HybridSequential(
        (0): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   

In [6]:
 def yolo2_forward(x, num_class, anchor_scales):
    """Transpose/reshape/organize convolution outputs."""
    stride = num_class + 5
    # transpose and reshape, 4th dim is the number of anchors
    x = x.transpose((0, 2, 3, 1))
    x = x.reshape((0, 0, 0, -1, stride))
    # now x is (batch, m, n, stride), stride = num_class + 1(object score) + 4(coordinates)
    # class probs
    cls_pred = x.slice_axis(begin=0, end=num_class, axis=-1)
    # object score
    score_pred = x.slice_axis(begin=num_class, end=num_class + 1, axis=-1)
    score = nd.sigmoid(score_pred)
    # center prediction, in range(0, 1) for each grid
    xy_pred = x.slice_axis(begin=num_class + 1, end=num_class + 3, axis=-1)
    xy = nd.sigmoid(xy_pred)
    # width/height prediction
    wh = x.slice_axis(begin=num_class + 3, end=num_class + 5, axis=-1)
    # convert x, y to positions relative to image
    x, y = transform_center(xy)
    # convert w, h to width/height relative to image
    w, h = transform_size(wh, anchor_scales)
    # cid is the argmax channel
    cid = nd.argmax(cls_pred, axis=-1, keepdims=True)
    # convert to corner format boxes
    half_w = w / 2
    half_h = h / 2
    left = nd.clip(x - half_w, 0, 1)
    top = nd.clip(y - half_h, 0, 1)
    right = nd.clip(x + half_w, 0, 1)
    bottom = nd.clip(y + half_h, 0, 1)
    output = nd.concat(*[cid, score, left, top, right, bottom], dim=4)
    return output, cls_pred, score, nd.concat(*[xy, wh], dim=4)


In [7]:
def transform_center(xy):
    """Given x, y prediction after sigmoid(), convert to relative coordinates (0, 1) on image."""
    b, h, w, n, s = xy.shape
    offset_y = nd.tile(nd.arange(0, h, repeat=(w * n * 1), ctx=xy.context).reshape((1, h, w, n, 1)), (b, 1, 1, 1, 1))
    # print(offset_y[0].asnumpy()[:, :, 0, 0])
    offset_x = nd.tile(nd.arange(0, w, repeat=(n * 1), ctx=xy.context).reshape((1, 1, w, n, 1)), (b, h, 1, 1, 1))
    # print(offset_x[0].asnumpy()[:, :, 0, 0])
    x, y = xy.split(num_outputs=2, axis=-1)
    x = (x + offset_x) / w
    y = (y + offset_y) / h
    return x, y


In [8]:
def transform_size(wh, anchors):
    """Given w, h prediction after exp() and anchor sizes, convert to relative width/height (0, 1) on image"""
    b, h, w, n, s = wh.shape
    aw, ah = nd.tile(nd.array(anchors, ctx=wh.context).reshape((1, 1, 1, -1, 2)), (b, h, w, 1, 1)).split(num_outputs=2, axis=-1)
    w_pred, h_pred = nd.exp(wh).split(num_outputs=2, axis=-1)
    w_out = w_pred * aw / w
    h_out = h_pred * ah / h
    return w_out, h_out

In [9]:
def yolo2_target(scores, boxes, labels, anchors, ignore_label=-1, thresh=0.5):
    """Generate training targets given predictions and labels."""
    b, h, w, n, _ = scores.shape
    anchors = np.reshape(np.array(anchors), (-1, 2))
    #scores = nd.slice_axis(outputs, begin=1, end=2, axis=-1)
    #boxes = nd.slice_axis(outputs, begin=2, end=6, axis=-1)
    gt_boxes = nd.slice_axis(labels, begin=1, end=5, axis=-1)
    target_score = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    target_id = nd.ones_like(target_score, ctx=scores.context) * ignore_label
    target_box = nd.zeros((b, h, w, n, 4), ctx=scores.context)
    sample_weight = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    for b in range(output.shape[0]):
        # find the best match for each ground-truth
        label = labels[b].asnumpy()
        valid_label = label[np.where(label[:, 0] > -0.5)[0], :]
        # shuffle because multi gt could possibly match to one anchor, we keep the last match randomly
        np.random.shuffle(valid_label)
        for l in valid_label:
            gx, gy, gw, gh = (l[1] + l[3]) / 2, (l[2] + l[4]) / 2, l[3] - l[1], l[4] - l[2]
            ind_x = int(gx * w)
            ind_y = int(gy * h)
            tx = gx * w - ind_x
            ty = gy * h - ind_y
            gw = gw * w
            gh = gh * h
            # find the best match using width and height only, assuming centers are identical
            intersect = np.minimum(anchors[:, 0], gw) * np.minimum(anchors[:, 1], gh)
            ovps = intersect / (gw * gh + anchors[:, 0] * anchors[:, 1] - intersect)
            best_match = int(np.argmax(ovps))
            target_id[b, ind_y, ind_x, best_match, :] = l[0]
            target_score[b, ind_y, ind_x, best_match, :] = 1.0
            tw = np.log(gw / anchors[best_match, 0])
            th = np.log(gh / anchors[best_match, 1])
            target_box[b, ind_y, ind_x, best_match, :] = mx.nd.array([tx, ty, tw, th])
            sample_weight[b, ind_y, ind_x, best_match, :] = 1.0
            # print('ind_y', ind_y, 'ind_x', ind_x, 'best_match', best_match, 't', tx, ty, tw, th, 'ovp', ovps[best_match], 'gt', gx, gy, gw/w, gh/h, 'anchor', anchors[best_match, 0], anchors[best_match, 1])
    return target_id, target_score, target_box, sample_weight


In [10]:
sce_loss = gluon.loss.SoftmaxCrossEntropyLoss(from_logits=False)
l1_loss = gluon.loss.L1Loss()

from mxnet import metric

class LossRecorder(mx.metric.EvalMetric):
    """LossRecorder is used to record raw loss so we can observe loss directly
    """
    def __init__(self, name):
        super(LossRecorder, self).__init__(name)

    def update(self, labels, preds=0):
        """Update metric with pure loss
        """
        for loss in labels:
            if isinstance(loss, mx.nd.NDArray):
                loss = loss.asnumpy()
            self.sum_metric += loss.sum()
            self.num_inst += 1

obj_loss = LossRecorder('objectness_loss')
cls_loss = LossRecorder('classification_loss')
box_loss = LossRecorder('box_refine_loss')


In [11]:
from mxnet import init
from mxnet import gpu

positive_weight = 3.0
negative_weight = 0.1
class_weight = 1.0
box_weight = 3.0

ctx = gpu(0)
net.collect_params().reset_ctx(ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.001, 'wd': 5e-4})



import time
from mxnet import autograd
for epoch in range(500):
    # reset data iterators and metrics
    train_data.reset()
    cls_loss.reset()
    obj_loss.reset()
    box_loss.reset()
    tic = time.time()
    for i, batch in enumerate(train_data):
        x = batch.data[0].as_in_context(ctx)
        y = batch.label[0].as_in_context(ctx)
        with autograd.record():
            x = net(x)
            output, cls_pred, score, xywh = yolo2_forward(x, num_class, scales)
            with autograd.pause():
                tid, tscore, tbox, sample_weight = yolo2_target(score, xywh, y, scales, thresh=0.5)
            # losses
            loss1 = sce_loss(cls_pred, tid, sample_weight * class_weight)
            score_weight = nd.where(sample_weight > 0,
                                    nd.ones_like(sample_weight) * positive_weight,
                                    nd.ones_like(sample_weight) * negative_weight)
            loss2 = l1_loss(score, tscore, score_weight)
            loss3 = l1_loss(xywh, tbox, sample_weight * box_weight)
            loss = loss1 + loss2 + loss3
        loss.backward()
        trainer.step(batch_size)
        # update metrics
        cls_loss.update(loss1)
        obj_loss.update(loss2)
        box_loss.update(loss3)

    print('Epoch %2d, train  %.5f,  %.5f,  %.5f time %.1f sec' % (
        epoch, cls_loss.get()[-1], obj_loss.get()[-1], box_loss.get()[-1], time.time()-tic))


Epoch  0, train  0.01416,  0.06110,  0.00557 time 84.3 sec
Epoch  1, train  0.01410,  0.06088,  0.00542 time 70.5 sec
Epoch  2, train  0.01407,  0.06070,  0.00531 time 73.4 sec
Epoch  3, train  0.01404,  0.06052,  0.00521 time 74.3 sec
Epoch  4, train  0.01396,  0.06030,  0.00511 time 70.1 sec
Epoch  5, train  0.01403,  0.06019,  0.00506 time 69.4 sec
Epoch  6, train  0.01393,  0.05996,  0.00497 time 68.3 sec
Epoch  7, train  0.01394,  0.05980,  0.00492 time 68.1 sec
Epoch  8, train  0.01388,  0.05960,  0.00485 time 68.8 sec
Epoch  9, train  0.01389,  0.05945,  0.00482 time 67.3 sec
Epoch 10, train  0.01381,  0.05923,  0.00476 time 65.2 sec
Epoch 11, train  0.01376,  0.05903,  0.00471 time 65.0 sec
Epoch 12, train  0.01374,  0.05885,  0.00469 time 65.0 sec
Epoch 13, train  0.01375,  0.05871,  0.00467 time 65.0 sec
Epoch 14, train  0.01372,  0.05851,  0.00463 time 65.3 sec
Epoch 15, train  0.01365,  0.05831,  0.00460 time 65.1 sec
Epoch 16, train  0.01363,  0.05814,  0.00458 time 64.9 s

In [25]:
def process_image(fname):
    with open(fname, 'rb') as f:
        im = image.imdecode(f.read())
    # resize to data_shape
    data = image.imresize(im, data_shape, data_shape)
    # minus rgb mean, divide std
    data = (data.astype('float32') - rgb_mean) / rgb_std
    # convert to batch x channel x height xwidth
    return data.transpose((2,0,1)).expand_dims(axis=0), im

def predict(x):
    x = net(x)
    output, cls_prob, score, xywh = yolo2_forward(x, num_class, scales)
    return nd.contrib.box_nms(output.reshape((0, -1, 6)))

x, im = process_image('D:/1/2.jpg')
out = predict(x.as_in_context(ctx))
print out.shape


(1L, 1568L, 6L)


In [26]:
#mpl.rcParams['figure.figsize'] = (6,6)
import matplotlib.pyplot as plt
colors = ['blue', 'green', 'red', 'black', 'magenta','yellow']

def box_to_rect(box, color, linewidth=3):
    """convert an anchor box to a matplotlib rectangle"""
    box = box.asnumpy()
    return plt.Rectangle(
        (box[0], box[1]), (box[2]-box[0]), (box[3]-box[1]),
        fill=False, edgecolor=color, linewidth=linewidth)


def display(im, out, threshold=0.5):
    plt.imshow(im.asnumpy())
    for row in out:
        row = row.asnumpy()
        class_id, score = int(row[0]), row[1]
        if class_id < 0 or score < threshold:
            continue
        color = colors[class_id%len(colors)]
        box = row[2:6] * np.array([im.shape[1],im.shape[0]]*2)
        rect = box_to_rect(nd.array(box), color, 2)
        plt.gca().add_patch(rect)
        text = class_names[class_id]
        plt.gca().text(box[0], box[1],
                       '{:s} {:.2f}'.format(text, score),
                       bbox=dict(facecolor=color, alpha=0.5),
                       fontsize=10, color='white')
    plt.show()

display(im, out[0], threshold=0.92)


In [24]:
#net.save_params('')
net.save_parameters('500epoch.params')